第６章までで説明してきたGLMは現実のデータ解析に応用できない。

\\( \because \\)実験・調査で得られたカウントデータのバラつきは、ポワソン分布や二項分布だけではうまく説明できない。全ての個体差を説明変数で表すことは不可能。

そこで以下を用いる。

**一般化線形混合モデル(generalized linear miexed model, GLMM)**「人間が測定できない・測定しなかった個体差」を組み込んだGLM。

複数の確率分布を部品とする。
* データのバラつきは二項分布・ポアソン分布
* 個体のバラつきは正規分布

というように表す。


## 7.1 例題：GLMでは説明できないカウントデータ

* 個体数：100
* 個体iごとの調査種子数：8個
* 生存していた種子数：\\( y_i \\)個
* 葉数：\\( x_i \in \{2,3,4,5,6\} \\)(各葉数ごとに20個体)

生存確率\\( q_i \\)を線形予測子とロジットリンク関数を組み合わせて、
$$ logit(q_i) = \beta_1 + \beta_2x_i $$

#### 補足：なぜ生存確率を線形予測子とロジットリンク関数の組み合わせとするか

二項分布のリンク関数はロジットリンク関数で表すと都合が良いため。

6.4節の要約。

ロジスティック関数

$$ q_i= logistic(z_i) = \frac{1}{1+exp(-z_i)} $$

変形すると

$$ log\frac{q_i}{1-q_i} = z_i $$

これがロジット関数(logit function)

$$ logit(q_i) = log\frac{q_i}{1-q_i} $$

これが線形予測子(LP)に等しいので、

$$  \begin{align}
\frac{q_i}{1-q_i} &= exp(LP)\\
&= exp(\beta_1+\beta_2x_i+\beta_3f_i)\\
&= exp(\beta_1)exp(\beta_2x_i)exp(\beta_3f_i)
 \end{align} $$
 
\\( \frac{q_i}{1-q_i} \\)はオッズと呼ばれ、(生存する確率)/(生存しない確率)。これが
 
* \\( exp(LP)\geqq 0 \\)なので推定計算に都合が良い
* 要因の効果が積で表されるのでわかりやすい

$$ p(y_i | \beta_1,\beta_2)=\binom{8}{y_i}q_i^{y_i}(1-q_i)^{8-y_i} $$

## 7.2 過分散と個体差

<code>
    > d <- read.csv("data.csv")
    > d4 <- d[d$x == 4,]
    > table(d4$y)
    0 1 2 3 4 5 6 7 8 
    3 1 4 2 1 1 2 3 3 
    > c(mean(d4$y), var(d4$y))
    [1] 4.050000 8.365789
[1] 4.050000 8.365789
</code>
生存種子数\\( y_i \\)が二項分布に従うなら、その分散は8*0.5*(1-0.5)=2くらいになるはずだが、8.36になっている。

-> このデータは二項分布と呼ぶにはバラつきが大きすぎ、二項分布では説明できない。これが**過分散**の状態

### 個体差が極端な例

生存種子数が0か8かというデータでやってみる。

<code>
    > d2 <-read.csv("data2.csv")
    > d24 <- d2[d2$x == 4,]
    > table(d24$y)
     0  8 
    10 10 
    > c(mean(d24$y), var(d24$y))
    [1]  4.00000 16.84211
</code>

上記と同様平均種子数は4だが分散は16まで増えている。

-> 全生存数/全調査種子数という割算をしても、これだけでは減少のパターンをうまく説明できない。

#### 個体差？場所差？

* 個体差：個体の遺伝子、年齢など生物的なもの
* 場所差：水分量や光環境のような生育環境の微妙な違いなど非生物的なもの

## 7.3 一般化線型混合モデル

一般化線型混合モデル(GLMM)は個体差や場所差の効果を考慮した統計モデル。

### 7.3.1 個体差を表すパラメーターの追加

個体iの個体差を表すパラメーター\\( r_i \\)を追加する

$$ logit(q_i) = \beta_1 + \beta_2x_i + r_i $$ 

GLMでは「観測されていない個体差はない」前提なので、\\( r_i = 0 \\)

\\( r_i > 0 \\)だと生存確率は高くなり、\\( r_i < 0 \\)だと低くなる。

### 7.3.2 個体差のばらつきを表す確率分布

GLMMの特徴：**個体差をあらわすパラメーター\\( \{r_1, r_2, ..., r_{100}\} \\)が何かの確率分布に従っていると仮定すること**

今回は個体差\\( r_i \\)は平均ゼロで標準偏差sの正規分布であるとする。（\\( r_i \\)は相互に独立）

確率密度関数\\( p(r_i | s) \\)は以下のとおり

$$ p(r_i | s) = \frac{1}{\sqrt{2\pi s^2}}exp\left(-\frac{r_i^2}{2s^2}\right) $$
### 7.3.3 線形予測子の構成要素：固定効果とランダム効果

線形予測子の構成要素を固定効果とランダム効果に分類。

一般化でない線形モデルでは、
* 固定効果：全体の平均を変える
* ランダム効果：平均は変化させないが全体のばらつきを変える

ex:このモデルの線形予測子\\( logit(q_i) = \beta_1 + \beta_2x_i + r_i \\)において、
* 固定効果：切片\\( \beta_1 \\)と端数の影響\\( \beta_2x_i \\)
* ランダム効果：個体差\\( r_i \\)

この分類はわかりにくいので、大域的パラメータ（データの広い範囲を説明）・局所的パラメータ（データのごく一部を説明）に分けたほうがいいかも（第10章で議論）

## 7.4 一般化線形混合モデルの最尤推定

個体差\\( r_i \\)は最尤推定できない。

\\( \because \\)100個の生存数データ\\( y_i \\)を説明するために100個のパラメータ\\( \{ \hat{r_1},\hat{r_2},...,\hat{r_{100}}\} \\)を最尤推定するとフルモデルになってしまう。

対処法：個体ごとの尤度\\( L_i \\)の式で\\( r_i \\)を積分してしまうことで\\( r_i \\)を削除する

$$ L_i = \int_{-\infty}^{\infty}p(y_i|\beta_1, \beta_2, r_i)p(r_i|s)dr_i $$

意味合い：色々な\\( r_i \\)の値における尤度を評価し、その期待値の算出をしている。

二項分布\\( p(y_i|\beta_1, \beta_2, r_i) \\)と正規分布\\( p(r_i|s) \\)をかけて\\( r_i \\)で積分することは**二種類の分布を混ぜていることに相当する**。

混合された分布は平均よりも分散が大きくなる過分散なモデルになる。

### 7.4.1 Rを使ってGLMMのパラメーターを推定
glmmMLを使用する。

<code> 
    library(glmmML)
    d <- read.csv("data.csv")
    glmmML(cbind(y, N -y) ~ x, data = d, family = binomial, cluster = id)+
    
    Call:  glmmML(formula = cbind(y, N - y) ~ x, family = binomial, data = d,      cluster = id) 
    
    
                  coef se(coef)      z Pr(>|z|)
    (Intercept) -4.190   0.8777 -4.774 1.81e-06
    x            1.005   0.2075  4.843 1.28e-06
    
    Scale parameter in mixing distribution:  2.408 gaussian 
    Std. Error:                              0.2202 
    
            LR p-value for H_0: sigma = 0:  2.136e-55 
    
    Residual deviance: 269.4 on 97 degrees of freedom       AIC: 275.4 
</code>

* coef：パラメーターの最尤推定値。\\( \beta_1 = -4.190 \\)(本だと-4.13、真の値は-4)、。\\( \beta_2 = -1.005 \\)(本だと-0.99、真の値は1)
* Scale parameter は「個体差\\( r_i \\)のばらつき」＝sの最尤推定値。真の値は3なので過小推定

このモデルの予測は第10章で行うが（階層ベイズモデル）、結果だけ見ると改善されている（図7.10参照）

## 7.5 現実のデータ解析にはGLMMが必要

GLMMのような考え方が必要になるかどうかのポイント：
* 同じ個体・場所から何度もサンプリングしているか
* 個体差や場所差が識別できてしまうようなデータのとり方をしているか

### 7.5.1 反復・疑似反復と統計モデルの関係

#### 各個体でひとつだけ種子の生死を調べている（1つの植木鉢に1個体）

このようなデータのとり方を**反復**という。

* 個体：反復
* 植木鉢：反復

個体差のばらつきも場所差もない。GLMMが使えないのでGLM。
$$ logit(q_i)=\beta_1 + \beta_2x_i $$

#### 各個体からN個の種子の生死を調べている（1つの植木鉢に1個体）

本章で扱ったもの。個体差のばらつきが生じる（この個体だけ全部死んでいる、など）
個体差がある場合は完全な反復ではなく**疑似反復**と呼ばれる。

* 個体：疑似反復
* 植木鉢：反復

$$ logit(q_i)=\beta_1 + \beta_2x_i + r_i $$

#### 各個体でひとつだけ種子の生死を調べている（1つの植木鉢に複数個体）

植木鉢差のばらつきが生じる（この植木鉢の個体は全部死んでいる、など）

* 個体：反復
* 植木鉢：疑似反復

$$ logit(q_i)=\beta_1 + \beta_2x_i + r_i $$

#### 各個体でN個の種子の生死を調べている（1つの植木鉢に複数個体）

個体差、植木鉢差のばらつきが生じる

* 個体：疑似反復
* 植木鉢：疑似反復

$$ logit(q_i)=\beta_1 + \beta_2x_i + r_i + r_j $$

階層ベイズモデルとMCMCを使っていく。

## 7.6 色々な分布のGLMM

### ポワソン分布

ポワソン分布と正規分布の無限混合分布

### 負の二項分布

ポワソン分布とガンマ分布の無限混合分布

### 正規分布かつ恒等リンク関数

線形混合モデルという。


## 7.7 この章のまとめと参考文献


* ここまでの章の例題のような架空データならば、簡単なGLMを使ってデータに見られるパターンを説明できたが、現実のデータではGLMがうまくあてはまらない場合がある(7.1例題:GLMでは説明できないカウントデータ)
* GLMでは「説明変数が同じならどの個体も均質」と仮定していたが、観測されていない個体差があるので、集団全体の生存種子数の分布は二項分布で期待されるより過分散なものになる(7.2過分散と個体差)
* このような状況に対応しているGLMMとは、線形予測子に個体差のばらつきをあらわすパラメーター\\( r_i \\)を追加し、全個体の\\( r_i \\)がある確率分布にしたがうと仮定した統計モデルである(7.3一般化線形混合モデル)
* 積分によって\\( r_i \\)を消去した尤度を最大化することで、GLMMの切片、傾きそして個体差\\( r_i \\)のばらつきといった、大域的なパラメーターを最尤推定できる(7.4一般化線形混合モデルの最尤推定)
* ひとつの個体から複数のデータをとったり、ひとつの場所に多数の調査対象がいるような状況は擬似反復とよばれ、このような構造のデータに統計モデルをあてはめるときには、個体差
* 場所差などをくみこんだGLMMが必要である(7.5現実のデータ解析にはGLMMが必要)
* データのばらつきをあらわす確率分布の種類がどのようなものであっても、個体差、場所差などに影響されるデータの部分集合があれば、これらの効果をランダム効果としてくみこんだ統計モデルで推定しなければならない(7.6いろいろな分布のGLMM)